<html>
    <h1 style="color:yellow;">MCDO-NN</h1>
</html>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import math

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

class MCDoNN:
    def __init__(self, input_dim, dropout_rate=0.2, hidden_layers=[64, 32, 16]):
        self.input_dim = input_dim
        self.dropout_rate = dropout_rate
        self.hidden_layers = hidden_layers
        self.model = self._build_model()
        self.scaler = MinMaxScaler()
        
    def _build_model(self):
        inputs = Input(shape=(self.input_dim,))
        x = inputs
        
        # Hidden layers with dropout and batch normalization
        for units in self.hidden_layers:
            x = Dense(units, activation='relu')(x)
            x = BatchNormalization()(x)
            x = Dropout(self.dropout_rate)(x)
        
        # Output layer
        outputs = Dense(1)(x)
        
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        return model
    
    def fit(self, X, y, validation_split=0.2, epochs=200, batch_size=32):
        # Scale the data
        X_scaled = self.scaler.fit_transform(X)
        
        # Early stopping callback
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True
        )
        
        # Train the model
        history = self.model.fit(
            X_scaled, y,
            validation_split=validation_split,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping],
            verbose=0
        )
        
        return history
    
    def predict_with_uncertainty(self, X, n_iterations=100):
        X_scaled = self.scaler.transform(X)
        predictions = []
        
        # Enable dropout at inference time
        for _ in range(n_iterations):
            pred = self.model(X_scaled, training=True)
            predictions.append(pred)
        
        # Calculate mean and standard deviation
        predictions = np.array(predictions).squeeze()
        mean_pred = np.mean(predictions, axis=0)
        std_pred = np.std(predictions, axis=0)
        
        return mean_pred, std_pred

def load_data():
    data = {
        'Commodity': [
            'Rice - Basmati', 'Rice (Other than Basmati)', 'Wheat', 'Other Cereals',
            'Pulses', 'Tobacco Unmanufactured', 'Tobacco Manufactured', 'Cashew',
            'Cashew Nut Shell Liquid', 'Sesame Seeds', 'Niger Seeds', 'Groundnut',
            'Other Oil Seeds', 'Vegetable Oils', 'Oil Meals', 'Guar Gum Meal',
            'Castor Oil', 'Shellac', 'Sugar', 'Molasses', 'Fruits / Vegetable Seeds',
            'Fresh Fruits', 'Fresh Vegetables', 'Processed Vegetables',
            'Processed Fruits and Juices', 'Cereal Preparations'
        ],
        '2018-2019': [2247.34, 1577.02, 20.84, 208.83, 155.13, 291.99, 200.98, 311.77,
                      2.44, 289.83, 5.11, 209.77, 49.37, 58.14, 591.9, 355.38, 445.41,
                      22.27, 490.83, 27.76, 78.52, 293.03, 425.53, 150.83, 299.22, 267.44],
        '2019-2020': [2032.1, 1021.77, 33.14, 97.22, 120.09, 274.61, 211.18, 281.3,
                      1.55, 254.97, 5.41, 190.32, 30.07, 65.97, 430.11, 268.09, 523.11,
                      21.7, 808.32, 36.7, 67.01, 277.68, 408.81, 152.54, 298.57, 273.45],
        '2020-2021': [2123.58, 1964.87, 108.77, 212.99, 154.76, 264.23, 157.66, 168.72,
                      1.03, 224.11, 12.02, 236.51, 35.35, 131.22, 461.65, 122.03, 438.01,
                      36.91, 1364.08, 44.49, 72.09, 230.65, 440.63, 213.87, 321.88, 280.21],
        '2021-2022': [1659.6, 2968.77, 630.15, 467.42, 135.25, 288.53, 174.91, 222.02,
                      1.89, 188.05, 4.77, 246.42, 14.04, 99.22, 471.65, 178.85, 615.62,
                      44.86, 1820.68, 95.4, 65.81, 301.3, 435.41, 213.77, 370.12, 314.92],
        '2022-2023': [2279.66, 3207.29, 1487.47, 524.85, 329.55, 446.72, 212.77, 157.09,
                      5.09, 242.87, 2.82, 245.57, 39.76, 178.9, 556.61, 343.65, 662.93,
                      48.8, 2649.0, 127.28, 73.03, 313.46, 439.52, 254.01, 440.62, 359.31]
    }
    return pd.DataFrame(data)

def prepare_data(df):
    X_list = []
    y_list = []
    commodities = []
    
    for commodity in df['Commodity']:
        values = df[df['Commodity'] == commodity].iloc[:, 1:].values.flatten()
        X = values[:-1].reshape(1, -1)  # Use all but last year as features
        y = values[-1]  # Use last year as target
        
        X_list.append(X)
        y_list.append(y)
        commodities.append(commodity)
    
    X = np.vstack(X_list)
    y = np.array(y_list)
    
    return X, y, commodities

def plot_predictions_mcdonn(df, predictions, uncertainties, commodities):
    fig = plt.figure(figsize=(15, 10))
    
    years = df.columns[1:].astype(str)
    next_year = '2023-2024'
    colors = plt.cm.rainbow(np.linspace(0, 1, len(commodities)))
    
    for idx, commodity in enumerate(commodities):
        values = df[df['Commodity'] == commodity].iloc[:, 1:].values.flatten()
        color = colors[idx]
        
        plt.plot(years, values, marker='o', color=color, alpha=0.5)
        plt.errorbar(next_year, predictions[idx], 
                    yerr=uncertainties[idx], 
                    fmt='o', capsize=5, color=color,
                    label=commodity)
    
    plt.title('MCDoNN Commodity Price Predictions with Uncertainty')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    
    return fig

def plot_evaluation_metrics_mcdonn(df, predictions, uncertainties, commodities):
    fig = plt.figure(figsize=(20, 15))
    
    # Get actual values (last year)
    actual_values = df.iloc[:, -1].values
    
    # 1. Prediction Error Distribution
    plt.subplot(2, 2, 1)
    errors = predictions - actual_values
    sns.histplot(errors, kde=True)
    plt.title('Prediction Error Distribution')
    plt.xlabel('Prediction Error')
    plt.ylabel('Frequency')
    
    # 2. Actual vs Predicted Scatter Plot
    plt.subplot(2, 2, 2)
    plt.scatter(actual_values, predictions, alpha=0.5)
    plt.plot([min(actual_values), max(actual_values)], 
             [min(actual_values), max(actual_values)], 'r--')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    r2 = r2_score(actual_values, predictions)
    plt.title(f'Actual vs Predicted (R² = {r2:.3f})')
    
    # Add commodity labels
    for i, commodity in enumerate(commodities):
        plt.annotate(commodity, (actual_values[i], predictions[i]), 
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    # 3. Uncertainty Distribution
    plt.subplot(2, 2, 3)
    sns.histplot(uncertainties, kde=True)
    plt.title('Uncertainty Distribution')
    plt.xlabel('Prediction Uncertainty (std)')
    plt.ylabel('Frequency')
    
    # 4. Uncertainty vs Error
    plt.subplot(2, 2, 4)
    plt.scatter(uncertainties, np.abs(errors), alpha=0.5)
    plt.xlabel('Prediction Uncertainty')
    plt.ylabel('Absolute Error')
    plt.title('Uncertainty vs Absolute Error')
    
    # Add commodity labels
    for i, commodity in enumerate(commodities):
        plt.annotate(commodity, (uncertainties[i], np.abs(errors[i])), 
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    plt.tight_layout()
    return fig

def main():
    # Load and prepare data
    df = load_data()
    X, y, commodities = prepare_data(df)
    
    # Create and train MCDoNN model
    model = MCDoNN(input_dim=X.shape[1])
    history = model.fit(X, y)
    
    # Make predictions with uncertainty
    mean_predictions, uncertainties = model.predict_with_uncertainty(X)
    
    # Calculate metrics
    rmse = math.sqrt(mean_squared_error(y, mean_predictions))
    mae = mean_absolute_error(y, mean_predictions)
    r2 = r2_score(y, mean_predictions)
    
    # Print metrics
    print("\nMCDoNN Model Evaluation Metrics")
    print("=" * 50)
    print(f"Overall RMSE: {rmse:.2f}")
    print(f"Overall MAE: {mae:.2f}")
    print(f"Overall R²: {r2:.3f}")
    
    # Print individual predictions
    print("\nPredictions for 2023-2024:")
    print("-" * 50)
    print(f"{'Commodity':<30} {'Prediction':>10} {'Uncertainty':>12}")
    print("-" * 50)
    for i, commodity in enumerate(commodities):
        print(f"{commodity:<30} {mean_predictions[i]:>10.2f} ±{uncertainties[i]:>10.2f}")
    
    # Create and save visualizations
    pred_fig = plot_predictions_mcdonn(df, mean_predictions, uncertainties, commodities)
    pred_fig.savefig('mcdonn_predictions.png', bbox_inches='tight', dpi=300)
    plt.close(pred_fig)
    
    eval_fig = plot_evaluation_metrics_mcdonn(df, mean_predictions, uncertainties, commodities)
    eval_fig.savefig('mcdonn_evaluation.png', bbox_inches='tight', dpi=300)
    plt.close(eval_fig)
    
    print("\nVisualization files have been saved:")
    print("1. mcdonn_predictions.png - Shows predictions with uncertainty bands")
    print("2. mcdonn_evaluation.png - Shows evaluation metrics and uncertainty analysis")

if __name__ == "__main__":
    main()


MCDoNN Model Evaluation Metrics
Overall RMSE: 1017.06
Overall MAE: 600.33
Overall R²: -0.520

Predictions for 2023-2024:
--------------------------------------------------
Commodity                      Prediction  Uncertainty
--------------------------------------------------
Rice - Basmati                       9.42 ±      1.75
Rice (Other than Basmati)           11.42 ±      2.29
Wheat                                5.28 ±      1.32
Other Cereals                        2.70 ±      1.07
Pulses                              -2.18 ±      1.41
Tobacco Unmanufactured              -0.00 ±      0.92
Tobacco Manufactured                -2.10 ±      1.44
Cashew                              -1.63 ±      1.28
Cashew Nut Shell Liquid             -3.99 ±      2.04
Sesame Seeds                        -1.42 ±      1.27
Niger Seeds                         -3.81 ±      1.94
Groundnut                           -0.50 ±      1.01
Other Oil Seeds                     -4.06 ±      1.88
Vegetable Oils     

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import math

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

class MultiHeadSelfAttention(Layer):
    def __init__(self, embed_dim, num_heads=8, dropout_rate=0.1):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate
        if embed_dim % num_heads != 0:
            raise ValueError(
                f'embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}'
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = Dense(embed_dim)
        self.key_dense = Dense(embed_dim)
        self.value_dense = Dense(embed_dim)
        self.combine_heads = Dense(embed_dim)
        self.dropout = Dropout(dropout_rate)
        
    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        weights = self.dropout(weights)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs, training=False):
        batch_size = tf.shape(inputs)[0]
        
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        
        attention, _ = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        output = self.dropout(output, training=training)
        return output

class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads, dropout_rate)
        self.ffn = Sequential([
            Dense(ff_dim, activation="relu"),
            Dropout(dropout_rate),
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout = Dropout(dropout_rate)

    def call(self, inputs, training=False):
        attention_output = self.att(inputs, training=training)
        out1 = self.layernorm1(inputs + attention_output)
        ffn_output = self.ffn(out1, training=training)
        return self.layernorm2(out1 + ffn_output)

class TransformerMCDO:
    def __init__(self, input_dim, embed_dim=32, num_heads=4, ff_dim=32, num_transformer_blocks=2, dropout_rate=0.1):
        self.input_dim = input_dim
        self.model = self._build_model(embed_dim, num_heads, ff_dim, num_transformer_blocks, dropout_rate)
        self.scaler = MinMaxScaler()
        
    def _build_model(self, embed_dim, num_heads, ff_dim, num_transformer_blocks, dropout_rate):
        inputs = Input(shape=(self.input_dim,))
        x = Dense(embed_dim)(inputs)
        x = Reshape((-1, embed_dim))(x)
        
        # Positional encoding
        positions = tf.range(start=0, limit=self.input_dim, delta=1)
        position_embeddings = tf.keras.layers.Embedding(input_dim=self.input_dim, output_dim=embed_dim)(positions)
        x = x + position_embeddings
        
        # Transformer blocks
        for _ in range(num_transformer_blocks):
            x = TransformerBlock(embed_dim, num_heads, ff_dim, dropout_rate)(x)
        
        x = GlobalAveragePooling1D()(x)
        x = Dropout(dropout_rate)(x)
        x = Dense(ff_dim, activation="relu")(x)
        x = Dropout(dropout_rate)(x)
        outputs = Dense(1)(x)
        
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        return model
    
    def fit(self, X, y, validation_split=0.2, epochs=200, batch_size=32):
        X_scaled = self.scaler.fit_transform(X)
        
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True
        )
        
        history = self.model.fit(
            X_scaled, y,
            validation_split=validation_split,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping],
            verbose=0
        )
        
        return history
    
    def predict_with_uncertainty(self, X, n_iterations=100):
        X_scaled = self.scaler.transform(X)
        predictions = []
        
        for _ in range(n_iterations):
            pred = self.model(X_scaled, training=True)
            predictions.append(pred)
        
        predictions = np.array(predictions).squeeze()
        mean_pred = np.mean(predictions, axis=0)
        std_pred = np.std(predictions, axis=0)
        
        return mean_pred, std_pred

# ... (keep the load_data and prepare_data functions from the previous implementation)

def plot_predictions_transformer(df, predictions, uncertainties, commodities):
    fig = plt.figure(figsize=(15, 10))
    
    years = df.columns[1:].astype(str)
    next_year = '2023-2024'
    colors = plt.cm.rainbow(np.linspace(0, 1, len(commodities)))
    
    for idx, commodity in enumerate(commodities):
        values = df[df['Commodity'] == commodity].iloc[:, 1:].values.flatten()
        color = colors[idx]
        
        plt.plot(years, values, marker='o', color=color, alpha=0.5)
        plt.errorbar(next_year, predictions[idx], 
                    yerr=uncertainties[idx], 
                    fmt='o', capsize=5, color=color,
                    label=commodity)
    
    plt.title('Transformer-MCDO Commodity Price Predictions with Uncertainty')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    
    return fig

def plot_attention_weights(model, X, commodities):
    """Plot attention weights for visualization"""
    # Get attention weights from the first transformer block
    attention_layer = model.model.layers[4]  # Adjust index based on your model
    attention_weights = attention_layer.att(tf.constant(X), training=False)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 15))
    axes = axes.ravel()
    
    for i in range(min(4, len(commodities))):
        sns.heatmap(attention_weights[i], ax=axes[i])
        axes[i].set_title(f'Attention Weights - {commodities[i]}')
    
    plt.tight_layout()
    return fig

def main():
    # Load and prepare data
    df = load_data()
    X, y, commodities = prepare_data(df)
    
    # Create and train Transformer-MCDO model
    model = TransformerMCDO(
        input_dim=X.shape[1],
        embed_dim=32,
        num_heads=4,
        ff_dim=32,
        num_transformer_blocks=2,
        dropout_rate=0.1
    )
    history = model.fit(X, y)
    
    # Make predictions with uncertainty
    mean_predictions, uncertainties = model.predict_with_uncertainty(X)
    
    # Calculate metrics
    rmse = math.sqrt(mean_squared_error(y, mean_predictions))
    mae = mean_absolute_error(y, mean_predictions)
    r2 = r2_score(y, mean_predictions)
    
    # Print metrics
    print("\nTransformer-MCDO Model Evaluation Metrics")
    print("=" * 50)
    print(f"Overall RMSE: {rmse:.2f}")
    print(f"Overall MAE: {mae:.2f}")
    print(f"Overall R²: {r2:.3f}")
    
    # Print individual predictions
    print("\nPredictions for 2023-2024:")
    print("-" * 50)
    print(f"{'Commodity':<30} {'Prediction':>10} {'Uncertainty':>12}")
    print("-" * 50)
    for i, commodity in enumerate(commodities):
        print(f"{commodity:<30} {mean_predictions[i]:>10.2f} ±{uncertainties[i]:>10.2f}")
    
    # Create and save visualizations
    pred_fig = plot_predictions_transformer(df, mean_predictions, uncertainties, commodities)
    pred_fig.savefig('transformer_mcdo_predictions.png', bbox_inches='tight', dpi=300)
    plt.close(pred_fig)
    
    # Plot attention weights
    att_fig = plot_attention_weights(model, X, commodities)
    att_fig.savefig('transformer_attention_weights.png', bbox_inches='tight', dpi=300)
    plt.close(att_fig)
    
    print("\nVisualization files have been saved:")
    print("1. transformer_mcdo_predictions.png - Shows predictions with uncertainty bands")
    print("2. transformer_attention_weights.png - Shows attention weight patterns")

if __name__ == "__main__":
    main()

NameError: name 'Sequential' is not defined